In [ ]:
!pip install geemap

In [ ]:
import os
import ee
import geemap

In [ ]:
ee.Authenticate(auth_mode='notebook')

In [ ]:
ee.Initialize(project='YOUR-GOOGLE-CLOUD-PROJECT-NAME')

In [ ]:
# DEFINE CITY AND DATE RANGE FOR ANALYSIS

# Prompt the user to enter the city
city_name = input("Enter your city name: ")

# Convert the city name to a valid variable name by removing spaces and hyphens
city_var = city_name.replace(" ", "")
city_var = city_var.replace("-", "")

# Define a dictionary to store FeatureCollections for the 13 cities
city_collections = {
    'Melbourne': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Melbourne'),
    'LosAngeles': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/LosAngeles'),
    'Helsinki': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Helsinki'),
    'MexicoCity': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/MexicoCity'),
    'Austin': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Austin'),
    'Minneapolis': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Minneapolis'),
    'Munich': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Munich'),
    'Turin': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Turin'),
    'PortoAlegre': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/PortoAlegre'),
    'Valencia': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Valencia'),
    'Belfast': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Belfast'),
    'Chennai': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Chennai'),
    'Vic': ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/urban_study_region/Vic'),
}

# Add the 'geo_type' property to each feature
def add_geo_type(feature):
    return feature.set('geo_type', feature.geometry().type())

# Map the function over each individual FeatureCollection in the dictionary
mapped_collections = {city: city_collection.map(add_geo_type) for city, city_collection in city_collections.items()}

# Type in your city here
city = mapped_collections.get(city_var)
aos_public_osm = ee.FeatureCollection('projects/YOUR-GOOGLE-CLOUD-PROJECT-NAME/assets/aos_public_osm/' + f'{city_var}_aos_public_osm')

# Filter for area greater than or equal to 1 ha
aos_public_osm_filtered_1ha = aos_public_osm.filter(ee.Filter.gte('aos_ha_pub', 1))

# Define analysis dates as the year from March 1st 2023 to March 1st 2024
start_date = '2023-03-01';
end_date = '2024-03-01';

# Define map
Map = geemap.Map()

# Get the geometry and bounding box
geometry = city.geometry();
bounding_box = geometry.bounds();

# Define visualisation
black_outline_vis = {
    'color': 'Black',
    'width': 2,
}

purple_outline_vis = {
    'color': 'Purple',
    'width': 2,
}

# Add layers to map
Map.addLayer(city, black_outline_vis, 'Urban Study Region')
Map.addLayer(aos_public_osm_filtered_1ha, purple_outline_vis, 'AOS Public 1ha OSM')

In [ ]:
# LANDSAT 8 IMAGERY CLOUD MASK

# Function to mask clouds from Landsat 8 Collection 2, Level 2 using the QA_PIXEL band (CFMask).
def maskl8clouds(image):
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturation_mask = image.select('QA_RADSAT').eq(0)

    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    return image.addBands(optical_bands, None, True) \
        .addBands(thermal_bands, None, True) \
        .updateMask(qa_mask) \
        .updateMask(saturation_mask)

# Map the function over one year of data
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .map(maskl8clouds)

In [ ]:
# SENTINEL MASK CLOUDS AND NDVI FUNCTION

# Function to mask clouds from Sentinel 2 imagery using the QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    # Return the masked and scaled data, without the QA bands
    return image.updateMask(mask).divide(10000).select(["B.*"]).copyProperties(image, ["system:time_start"])

# Calculate NDVI
def calculate_ndvi(image):
    nir = image.select('B8')  # Near-infrared band
    red = image.select('B4')  # Red band
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
# NDVI CALCULATION - ANNUAL AVERAGE - SENTINEL

# Load Sentinel-2 TOA reflectance data using dates defined earlier in this notebook
sentinel_collection_annual_average = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(bounding_box) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)) \
    .map(mask_s2_clouds)

# Apply NDVI calculation to the image collection
annual_average_ndvi_collection = sentinel_collection_annual_average.map(calculate_ndvi)

# Calculate the overall average NDVI for all 12 months of the year
annual_average_ndvi = annual_average_ndvi_collection.select('NDVI').mean()

# Clip the NDVI image to the areas of open space to save processing time
annual_average_ndvi_image_clipped = annual_average_ndvi.clip(aos_public_osm_filtered_1ha)

# Clip the NDVI image to the city boundary
annual_average_ndvi_image_clipped_city = annual_average_ndvi.clip(city)

# Create a binary mask greater than or equal to NDVI 0.2
binary_ndvi = annual_average_ndvi_image_clipped_city.gte(0.2).rename('Binary_NDVI');

# Visualise the raster where white is NDVI greater than or equal to 0.2 and black is not
binary_ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['White','Black'],
}

# Define visualisation parameters for overall NDVI visualisation
ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['GhostWhite', 'PaleGreen', 'LightGreen', 'Green', 'ForestGreen']
}

# Add to map
Map.addLayer(annual_average_ndvi_image_clipped, ndvi_vis, 'NDVI - Annual Average')
Map.addLayer(binary_ndvi, binary_ndvi_vis, 'Binary NDVI');

In [ ]:
# DEFINE LARGE PUBLIC URBAN GREEN SPACE (LPUGS)

# LPUGS defined as NDVI 0.2 - 1
min_NDVI = 0.2
max_NDVI = 1

# Define mask based on max and min NDVI thresholds
lpugs_areas_annual_average = annual_average_ndvi_image_clipped.gte(min_NDVI).And(annual_average_ndvi_image_clipped.lte(max_NDVI))

# Apply mask
lpugs_areas_annual_average_clip = lpugs_areas_annual_average.updateMask(
    lpugs_areas_annual_average.gte(min_NDVI).And(lpugs_areas_annual_average.lte(max_NDVI))
)

# Define visualisation parameters and add to map
lpugs_vis = {
    'min': 0.2,
    'max': 1,
    'palette': ['Red'],
}

Map.addLayer(lpugs_areas_annual_average_clip, lpugs_vis, 'LPUGS - Annual Average NDVI')

In [ ]:
# ADD NDVI ATTRIBUTES TO EACH AOS_PUBLIC_OSM

def add_ndvi_to_feature(feature, image, name='NDVI'):
    # Get the geometry of the feature
    geometry = feature.geometry()

    # Use reduceRegion to calculate the mean NDVI value within each polygon
    ndvi_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=10
    ).get('NDVI')

    # Set the calculated rounded NDVI as a property of the feature
    return feature.set({name: ndvi_value})

def calculate_ndvi_area(feature, image, name='LPUGS_area'):   
    geometry = feature.geometry()
    ndvi_area = image.multiply(ee.Image.pixelArea()).rename('Area')
    ndvi_area_calc = ndvi_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=10,
        maxPixels=1e10
    )
    ndvi_area_m2 = ee.Number(ndvi_area_calc.get('Area'))
    ndvi_area_ha = ndvi_area_m2.divide(1e4)
    
    # Set the calculated LPUGS area as a property of the feature
    return feature.set({name: ndvi_area_ha})

# Usage example
attribute_name_ndvi_aa = 'NDVI_aa_av'
attribute_name_lpugs_area_aa = 'LPUGS_aa_ha'

with_annual_average_ndvi = aos_public_osm_filtered_1ha.map(lambda feature: add_ndvi_to_feature(feature, annual_average_ndvi_image_clipped, name=attribute_name_ndvi_aa))

aos_public_osm_lpugs = with_annual_average_ndvi.map(lambda feature: calculate_ndvi_area(feature, lpugs_areas_annual_average_clip, name=attribute_name_lpugs_area_aa))

# Filter for NDVI greater than or equal to 0.2
aos_public_osm_lpugs_ndvi_0point2 = aos_public_osm_lpugs.filter(ee.Filter.gte('NDVI_aa_av', 0.2))

In [ ]:
# FILTER LAYER TO ONLY POLYGONS

# Define a function to get the geometry type for a feature
def get_geometry_type(feature):
    # Create a dummy property 'geometryType' and set it to the geometry type
    return feature.set('geometryType', ee.String(feature.geometry().type()))

# Map the function over the FeatureCollection
mapped_features = aos_public_osm_lpugs_ndvi_0point2.map(get_geometry_type)

# Extract the 'geometryType' property as a list
geometry_types_list = mapped_features.aggregate_array('geometryType')

# Filter the features to exclude LineString on the server side
filtered_features = mapped_features.filter(
    ee.Filter.Or(
        ee.Filter.eq('geometryType', 'Polygon'),
        ee.Filter.eq('geometryType', 'MultiPolygon'),
        ee.Filter.eq('geometryType', 'LinearRing')
    )
)

# Create a new FeatureCollection with only non-LineString features on the server side
lpugs_final = ee.FeatureCollection(filtered_features)

In [ ]:
# SHOW MAP

Map.centerObject(city);
Map

In [ ]:
# EXPORT LPUGS POLYGONS

task = ee.batch.Export.table.toDrive(
    collection=lpugs_final,
    description='Export LPUGS Polygons to Drive ' + f'{city_var}',
    fileNamePrefix='LPUGS_Polygons_' + f'{city_var}_2025',
    fileFormat='SHP',
    selectors=['fid', 'aos_id', 'aos_ha_pub', 'NDVI_aa_av', 'LPUGS_aa_ha']
)
task.start()

# EXPORT NDVI RASTER

task = ee.batch.Export.image.toDrive(
    image=binary_ndvi,
    description='Export NDVI Image to Drive ' + f'{city_var}',
    fileNamePrefix='NDVI_0.2_' + f'{city_var}_2025',
    fileFormat='GeoTIFF',
    crs='EPSG:4326',
    scale=10,
    region=geometry,
    maxPixels=1e13,
)
task.start()

# Open your Google Drive to find the exported files ready for local download